In [1]:
import os

os.makedirs("/Users/tongaimutengwa/Projects/homework5", exist_ok=True)
home = "/Users/tongaimutengwa/Projects/homework5"
os.chdir(home)
!pwd

/Users/tongaimutengwa/Projects/homework5


In [2]:
import subprocess

try:
    # Run the uv --version command and capture the output
    result = subprocess.run(
        ['uv', '--version'], 
        capture_output=True, 
        text=True, 
        check=True
    )
    # The version is usually the first line of the output
    print(f"UV version: {result.stdout.strip()}")
except FileNotFoundError:
    print("Error: The 'uv' command was not found in your system's PATH.")
except subprocess.CalledProcessError as e:
    print(f"Error running uv: {e.stderr.strip()}")

UV version: uv 0.9.6 (265224465 2025-10-29)


In [3]:
#!curl -O https://files.pythonhosted.org/packages/py3/s/scikit_learn/scikit_learn-1.6.1-py3-none-any.whl

In [4]:
!ls

Dockerfile                          pipeline_v1.bin
get-docker.sh                       requirements.txt
homework5.ipynb                     scikit_learn-1.6.1-py3-none-any.whl
homework5.py


In [5]:
import hashlib

# Define the file paths
# Filter out all paths that are directories
# Define the file paths
file_path = os.listdir('.')
# Filter out all paths that are directories
actual_files = [path for path in file_path if os.path.isfile(path)]

# Compute and print the SHA256 hash for each file
for path in actual_files:
    with open(path, "rb") as f:
        file_bytes = f.read()
        hash_value = hashlib.sha256(file_bytes).hexdigest()
        
        # FIX IS HERE: Use 'path' (the string filename), not 'file_bytes' (the binary content)
        print(f"✅ SHA256 hash for {path.split('/')[-1]}:") 
        print("sha256:" + hash_value + "\n")

✅ SHA256 hash for pipeline_v1.bin:
sha256:c85a57297c7b23f1afee8fce787d1a1a64633484ffb98e62fc8fb35406b891aa

✅ SHA256 hash for requirements.txt:
sha256:a836b1cd655cb8a41b8bf428d9ea2e41049926362e0a68f3338f22b95eaba23c

✅ SHA256 hash for get-docker.sh:
sha256:8f2eae8097b281cac4f4f7362b690c393016d62a960403f751f7d0fac238e964

✅ SHA256 hash for Dockerfile:
sha256:8d0aa8bd6ed6b12960015143f3116aa04ae6c2054c8730d72894e57fcfbd9aa2

✅ SHA256 hash for scikit_learn-1.6.1-py3-none-any.whl:
sha256:e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855

✅ SHA256 hash for homework5.ipynb:
sha256:5146d169749954de244d52c133756df847e3735084b77fc23ec19567b7c7f038

✅ SHA256 hash for homework5.py:
sha256:1dd44dd85946b91df962c75ecd0d39546338d85a247a3a41e884aaade6267996



In [6]:
#!wget https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin

In [7]:
ls -la

total 104
drwxr-xr-x@ 10 tongaimutengwa  staff    320 Oct 30 11:03 ./
drwxr-xr-x@ 19 tongaimutengwa  staff    608 Oct 30 12:31 ../
drwxr-xr-x@  8 tongaimutengwa  staff    256 Oct 30 09:30 .venv/
-rw-r--r--@  1 tongaimutengwa  staff    458 Oct 30 11:16 Dockerfile
-rw-r--r--@  1 tongaimutengwa  staff  21013 Oct 30 10:25 get-docker.sh
-rw-r--r--@  1 tongaimutengwa  staff   9673 Oct 30 10:52 homework5.ipynb
-rw-r--r--@  1 tongaimutengwa  staff    568 Oct 30 10:56 homework5.py
-rw-r--r--@  1 tongaimutengwa  staff   1300 Oct 30 10:14 pipeline_v1.bin
-rw-r--r--@  1 tongaimutengwa  staff    406 Oct 30 12:02 requirements.txt
-rw-r--r--@  1 tongaimutengwa  staff      0 Oct 30 09:33 scikit_learn-1.6.1-py3-none-any.whl


In [8]:
import pickle

with open("pipeline_v1.bin", "rb") as f_in:
    dv, model = pickle.load(f_in)

In [9]:
record = {
    "lead_source": "paid_ads",
    "number_of_courses_viewed": 2,
    "annual_income": 79276.0
}

X = dv.transform([record])

y_pred = model.predict_proba(X)[:, 1]

print(f"Prediction probability: {y_pred[0]:.4f}")

Prediction probability: 0.5336


In [10]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle

# Load the model and vectorizer
with open("pipeline_v1.bin", "rb") as f_in:
    dv, model = pickle.load(f_in)

# Define the input data model
class Client(BaseModel):
    lead_source: str
    number_of_courses_viewed: int
    annual_income: float

# Create the FastAPI app
app = FastAPI()

@app.post("/predict")
def predict(client: Client):
    record = client.model_dump()
    X = dv.transform([record])
    y_pred = model.predict_proba(X)[:, 1]
    return {"prediction_probability": y_pred[0]}

# Run fastapi server Subtask: Start the FastAPI application using uvicorn in a way that it runs in the background or in a separate thread

In [ ]:
import threading
import time
import uvicorn

def run_server():
    """Runs the uvicorn server."""
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Create and start the server thread
server_thread = threading.Thread(target=run_server)
server_thread.start()

# Allow the server to initialize
time.sleep(1)

print("FastAPI server is running in a separate thread.")

In [11]:
import requests

url = "http://localhost:8000/predict"  # FastAPI app is running on localhost:8000
client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}

response = requests.post(url, json=client)
print(response.json())

{'prediction_probability': 0.5340417283801275}


In [12]:
!docker images agrigorev/zoomcamp-model:2025

REPOSITORY                 TAG       IMAGE ID       CREATED      SIZE
agrigorev/zoomcamp-model   2025      14d79fde0bbf   8 days ago   181MB


In [13]:
import requests

# 1. Define the URL for your local running container
url = "http://127.0.0.1:8000/predict"

# 2. Define the client data to be scored
client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}

# 3. Send the POST request
response = requests.post(url, json=client)
data = response.json()

# 4. Extract the probability using the CORRECT key
probability = data['prediction_probability']

print(f"The probability that this lead will convert is: {probability}")

The probability that this lead will convert is: 0.5340417283801275
